In [1]:
import vectorbt as vbt
import numpy as np
import pandas as pd
import yfinance as yf
from numba import njit
import ta

In [2]:
# Data
symbol = 'SPXL'  # Using QQQ as an example

strat_name = 'dual_ema_crossover'

freq='1D'

price = vbt.YFData.download(
    symbol,
    start='1/1/2023',
    interval='1d'
).get('Close')

fast_level = np.arange(9,27)
slow_level = np.arange(26,60)
EMA = vbt.IndicatorFactory.from_ta('EMAIndicator')
# ema13 = EMA.run(price, 5)
# ema48 = EMA.run(price, 10)

windows = np.arange(2, 101)
ema13, ema48 = EMA.run_combs(price, window=windows, r=2, short_names=['ema_fast', 'ema_slow'])

entries = ema13.ema_indicator_crossed_above(ema48)
exits = ema48.ema_indicator_crossed_below(ema48)

@njit
def adjust_sl_func_nb(c):
    current_profit = (c.val_price_now - c.init_price) / c.init_price
    if current_profit >= 0.6:
        return 0.4, True
    if current_profit >= 0.40:
        return 0.05, True
    elif current_profit >= 0.2:
        return 0.01, True
    # elif current_profit >= 0.05:
    #     return 0.01, True
    # else:
    #     return 0.2, False
    return c.curr_stop, c.curr_trail

take_profit_pct = 20  # 10%
stop_loss_pct = 0.1  # -5%

# Backtest with vectorbt, specifying short entries and exits
pf = vbt.Portfolio.from_signals(
    close=price,
    entries=entries,
    exits=exits,
    short_entries=exits,
    short_exits=entries,
    # init_cash=10000,
    # size=1,
    # size_type=vbt.portfolio.enums.SizeType.Percent,
    sl_stop=stop_loss_pct,
    adjust_sl_func_nb=adjust_sl_func_nb,
    tp_stop=take_profit_pct,
    freq=freq
)

In [3]:
pf.total_return()[::-1]

ema_fast_window  ema_slow_window
99               100                0.000000
98               100                0.000000
                 99                 0.000000
97               100                0.000000
                 99                 0.000000
                                      ...   
2                7                  0.577342
                 6                  0.496614
                 5                  0.456877
                 4                  0.503390
                 3                  0.705587
Name: total_return, Length: 4851, dtype: float64

In [4]:
xx = pf.get_orders()

# pf.cumulative_returns().vbt.plot().show()
max_pf = (2, 5) # pf.total_return().idxmax()


In [5]:
max_pf

(2, 5)

In [6]:
feats = list(pf.total_return().index.names)

In [7]:
import json
best_feats = dict({f:int(m) for f,m in zip(feats, max_pf)})

In [8]:
best_feats

{'ema_fast_window': 2, 'ema_slow_window': 5}

In [9]:

opf = pf[(2,5)]
opf.plot().show()

# Show performance metrics
print(max_pf)


print(opf.stats())


(2, 5)
Start                         2023-01-03 05:00:00+00:00
End                           2024-04-12 04:00:00+00:00
Period                                321 days 00:00:00
Start Value                                       100.0
End Value                                    145.687656
Total Return [%]                              45.687656
Benchmark Return [%]                         104.938796
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              32.791902
Max Drawdown Duration                 122 days 00:00:00
Total Trades                                          8
Total Closed Trades                                   7
Total Open Trades                                     1
Open Trade PnL                               -12.046531
Win Rate [%]                                  57.142857
Best Trade [%]                                33.957467
Worst Trade [%]                          

In [10]:
# Make dataframe to store 👉🏽 `strategy_stats`
base_stats = opf.stats()

In [11]:
base_stats['Features'] = best_feats
base_stats['Ticker'] = symbol

In [12]:
df_store = pd.DataFrame(base_stats).T.reset_index(drop=True).convert_dtypes()

In [13]:
import re

def to_snake_case(df):
    """
    Convert the column names of a DataFrame to lowercase snake_case format,
    removing special characters and newlines before conversion.
    
    Parameters:
    - df: pandas.DataFrame
    
    Returns:
    - DataFrame with column names in snake_case format.
    """
    # First, replace newlines with spaces, then remove special characters except spaces
    df.columns = [col.replace('\n', '') for col in df.columns]
    df.columns = [col.replace('[%]', '') for col in df.columns]
    df.columns = [col.strip() for col in df.columns]
    df.columns = [col.replace(' ', '_') for col in df.columns]
    df.columns = [col.lower() for col in df.columns]
    
    return df

In [14]:
df_store_final = to_snake_case(df_store)

In [15]:
from data_process.store_strategy import ProcessData

In [16]:
p = ProcessData()

In [17]:
df_store_final.columns

Index(['start', 'end', 'period', 'start_value', 'end_value', 'total_return',
       'benchmark_return', 'max_gross_exposure', 'total_fees_paid',
       'max_drawdown', 'max_drawdown_duration', 'total_trades',
       'total_closed_trades', 'total_open_trades', 'open_trade_pnl',
       'win_rate', 'best_trade', 'worst_trade', 'avg_winning_trade',
       'avg_losing_trade', 'avg_winning_trade_duration',
       'avg_losing_trade_duration', 'profit_factor', 'expectancy',
       'sharpe_ratio', 'calmar_ratio', 'omega_ratio', 'sortino_ratio',
       'features', 'ticker'],
      dtype='object')

In [18]:
p.store_df(df_store_final)

2024-04-13 14:10:54,733:INFO - HTTP Request: POST https://lkxrvqyovxhuftwaggcw.supabase.co/rest/v1/strategy_stats "HTTP/1.1 201 Created"


"An error occurred: 'APIResponse[TypeVar]' object has no attribute 'error'"

In [19]:
XXX

NameError: name 'XXX' is not defined

In [ ]:
df_store

In [ ]:
to_snake_case(df_store)

In [ ]:
df_store.to_csv('strategy_stats.csv')

In [ ]:
df_store

In [ ]:
xx.config['close'][max_pf]

In [ ]:
pf_kwargs = dict(size=np.inf, fees=0.001, freq='1D')

fig = pf.total_return().vbt.heatmap(
    x_level='fast_ema', y_level='slow_ema', symmetric=True,
    trace_kwargs=dict(colorbar=dict(title='Total return', tickformat='%')))
fig.show()